In [2]:
import pandas as pd

Load the Datasets

In [3]:
customersDf = pd.read_csv("Customers.csv")
productsDf = pd.read_csv("Products.csv")
transactionsDf = pd.read_csv("Transactions.csv")

Display first few Row

In [4]:
print("Customers: ")
display(customersDf.head())
print("\nProducts: ")
display(productsDf.head())
print("\nTransactions: ")
display(transactionsDf.head())

Customers: 


,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15



Products: 


,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31



Transactions: 


,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


View Basic Info

In [5]:
print(customersDf.info())
print(productsDf.info())
print(transactionsDf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    100 non-null    object 
 1   ProductName  100 non-null    object 
 2   Category     100 non-null    object 
 3   Price        100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------

Storing Data Into MYSQL

In [6]:
# Pre-req
!pip install mysql-connector-python sqlalchemy

In [7]:
# Libs
import mysql.connector
from sqlalchemy import create_engine

In [13]:
# Login
DbUser = 'root'
DbPassword = '?????'
DbHost = 'localhost'
DbPort = '3306'
DbName = 'eCommerceDb'

# Engine
engine = create_engine(f"mysql+mysqlconnector://{DbUser}:{DbPassword}@{DbHost}:{DbPort}/{DbName}")

# Connection
conn = mysql.connector.connect(
    host = DbHost,
    user = DbUser,
    password = DbPassword
)

# Create Database, since it doesn't exist
cursor = conn.cursor()
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DbName}")
cursor.close()
conn.close()

Working in SQL

In [14]:
conn = mysql.connector.connect(
    host = DbHost,
    user = DbUser,
    password = DbPassword,
    database = DbName
)

cursor = conn.cursor()

# Create Customers Table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Customers (
        CustomerID VARCHAR(10) PRIMARY KEY,
        CustomerName VARCHAR(255),
        Region VARCHAR(255),
        SignupDate DATETIME
    )
""")

# Create Products Table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Products (
        ProductID VARCHAR(10) PRIMARY KEY,
        ProductName VARCHAR(255),
        Category VARCHAR(255),
        Price DECIMAL(10,2)
    )
""")

# Create Transactions Table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Transactions (
        TransactionID VARCHAR(10) PRIMARY KEY,
        CustomerID VARCHAR(10),
        ProductID VARCHAR(10),
        TransactionDate DATETIME,
        Quantity INT,
        TotalValue DECIMAL(10,2),
        Price DECIMAL(10,2),
        FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
        FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
    )
""")

conn.commit()
cursor.close()
conn.close()

print("Tables created")

Tables created


In [15]:
# Converted DateTime Strings into DateTimes
customersDf['SignupDate'] = pd.to_datetime(customersDf['SignupDate'], format="%Y-%m-%d")
transactionsDf['TransactionDate'] = pd.to_datetime(transactionsDf['TransactionDate'], format="%Y-%m-%d %H:%M:%S")

In [16]:
print(f"Connecting to: {DbUser}@{DbHost}:{DbPort}/{DbName}")

Connecting to: root@localhost:3306/eCommerceDb


In [21]:
from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS transactions;"))
    conn.execute(text("DROP TABLE IF EXISTS customers;"))
    conn.execute(text("DROP TABLE IF EXISTS products;"))

customersDf.to_sql(name='customers', con=engine, if_exists='replace', index=False)
productsDf.to_sql(name='products', con=engine, if_exists='replace', index=False)
transactionsDf.to_sql(name='transactions', con=engine, if_exists='replace', index=False)

print("Data inserted successfully!")

Data inserted successfully!


In [25]:
query = """
SELECT 
    t.TransactionID, 
    t.CustomerID, 
    c.CustomerName, 
    c.Region, 
    c.SignupDate, 
    t.ProductID, 
    p.ProductName, 
    p.Category, 
    p.Price AS ProductPrice, 
    t.TransactionDate, 
    t.Quantity, 
    t.TotalValue 
FROM Transactions t
JOIN Customers c ON t.CustomerID = c.CustomerID
JOIN Products p ON t.ProductID = p.ProductID;
"""

# Read data into Pandas DataFrame
master_df = pd.read_sql(query, con=engine)

# Display first few rows to verify
display(master_df.head())

# Store into a new table in MySQL
master_df.to_sql(name='master_transactions', con=engine, if_exists='replace', index=False)

print("master_transactions table created successfully!")

,TransactionID,CustomerID,CustomerName,Region,SignupDate,ProductID,ProductName,Category,ProductPrice,TransactionDate,Quantity,TotalValue
0,T00436,C0001,Lawrence Carroll,South America,2022-07-10,P029,TechPro Headphones,Electronics,433.64,2024-11-02 17:04:16,3,1300.92
1,T00445,C0001,Lawrence Carroll,South America,2022-07-10,P083,ActiveWear Smartwatch,Electronics,455.72,2024-05-07 03:11:44,2,911.44
2,T00085,C0001,Lawrence Carroll,South America,2022-07-10,P096,SoundWave Headphones,Electronics,307.47,2024-04-08 00:01:00,2,614.94
3,T00932,C0001,Lawrence Carroll,South America,2022-07-10,P022,HomeSense Wall Art,Home Decor,137.54,2024-09-17 09:01:18,3,412.62
4,T00015,C0001,Lawrence Carroll,South America,2022-07-10,P054,SoundWave Cookbook,Books,57.30,2024-01-19 03:12:55,2,114.60


master_transactions table created successfully!
